In [110]:
import ftplib
import os
import numpy as np
import pandas as pd

#data_path = '/home/drew/dev/spp-da-price-forecast/data/'
data_path = 'data/'


In [83]:
def getData(ftp,path,fpath):
    for pa in fpath:
        ftp.cwd(pa)
    filenames = ftp.nlst()

    for filename in filenames:
        host_file = os.path.join(path, filename)
        if not host_file.endswith('0800.csv'):
            if "." not in host_file:
                try:
                    ftp = getData(ftp,path,[filename])
                    
                except OSError as exc: 
                        print(exc)
                        raise
        else:
            try:
                with open(host_file, 'wb') as local_file:
                    ftp.retrbinary('RETR ' + filename, local_file.write)
            except Exception as e:
                    print str(e)
    ftp.cwd('..')
    return ftp

In [102]:
def getWind(year):
  if not os.path.exists(data_path + 'DAWF/'+year):
    print("mdir " + data_path + 'DAWF/'+year)
    os.makedirs(data_path + 'DAWF/'+year)
    
  ftp = ftplib.FTP('pubftp.spp.org')
  fpath = ['Operational_Data', 'DAWF', year]
  ftp.login()
  ftp = getData(ftp, data_path + 'DAWF/'+year, fpath)
  ftp.quit()

In [103]:
def getMTLF(year):
  if not os.path.exists(data_path + 'MTLF/'+year):
    print("mdir " + data_path + 'MTLF/'+year)
    os.makedirs(data_path + 'MTLF/'+year)
    
  ftp = ftplib.FTP('pubftp.spp.org')
  fpath = ['Operational_Data', 'MTLF', year]
  ftp.login()
  ftp = getData(ftp, data_path + 'MTLF/'+year, fpath)
  ftp.quit()

In [106]:
getWind('2016')
getMTLF('2016')


mdir data/MTLF/2016


In [105]:
#getWind('2017')
getMTLF('2017')

mdir data/MTLF/2017


In [178]:
def convertWind(fname):
    col_names = ['Interval', 'GMTIntervalEnd', 'Forecast MW']
    wf = pd.read_csv(fname, header=0, names=col_names)
    df = pd.DataFrame(wf)
    df['Interval'] = pd.to_datetime(df['Interval'])
    df['dateminustime'] = df['Interval'].apply(lambda df : pd.datetime(year=df.year, month=df.month, day=df.day))
    df.set_index(df['dateminustime'],inplace=True)
    df.sort()
    days = list(set(r for r in df['dateminustime']))
    days.sort()

    with open('sppwind.csv', 'a') as out:
        for index, row in df.iterrows():
            if row['dateminustime'] == days[2]:
              out.write(str(row['Interval']) + ',' + str(row['Forecast MW']) + '\n') #header

In [179]:
def convertMTLF(fname):
    col_names = ['Interval', 'GMTIntervalEnd', 'MTLF', 'Averaged Actual']
    wf = pd.read_csv(fname, header=0, names=col_names)
    df = pd.DataFrame(wf)
    df['Interval'] = pd.to_datetime(df['Interval'])
    df['dateminustime'] = df['Interval'].apply(lambda df : pd.datetime(year=df.year, month=df.month, day=df.day))
    df.set_index(df['dateminustime'],inplace=True)
    df.sort()
    days = list(set(r for r in df['dateminustime']))
    days.sort()

    with open('sppmtlf.csv', 'a') as out:
        for index, row in df.iterrows():
            if row['dateminustime'] == days[2]:
              out.write(str(row['Interval']) + ',' + str(row['MTLF']) + '\n') #header

In [180]:
winddir = "data/DAWF/"
for root, subdirs, files in os.walk(winddir):
  subdirs.sort()
  for sd in subdirs:
    for sroot, ssbdirs, sfile in os.walk(os.path.join(root,sd)):
      sfile.sort()
      for f in sfile:
        convertWind(str(os.path.join(sroot,f)))
      

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:8: FutureWarning: sort(....) is deprecated, use sort_index(.....)


In [181]:
mtlfdir = "data/MTLF/"
for root, subdirs, files in os.walk(mtlfdir):
  subdirs.sort()
  for sd in subdirs:
    for sroot, ssbdirs, sfile in os.walk(os.path.join(root,sd)):
      sfile.sort()
      for f in sfile:
        convertMTLF(os.path.join(sroot,f))
      

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:8: FutureWarning: sort(....) is deprecated, use sort_index(.....)
